In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
import scipy.io

%load_ext autoreload
%autoreload 2
%matplotlib inline

#this file has all my utility functions -- although I'm reproducing the important ones below
from glm_utils import *

/Users/cocjinjb/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/cocjinjb/anaconda/lib/python2.7/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/Users/cocjinjb/anaconda/lib/python2.7/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [2]:
#magic numbers 6 and 9 come from cross validation to find best possible lag and offet. 
#Note that was done for the natural 
#scenes stimulus so might not be the best here.

def arrange_data_glm(dff_traces, images, stim_table):
	#declare a dictionary of empty lists for each cell trace, 
	#and a list for the stimulus
	data = []
	stim_array = []
	im_array = []

	#average each trace over the presentation of each stimulus
	for index, row in stim_table.iterrows():
	    stim_array.append(images[row['frame']])
	    im_array.append(row['frame'])

	    try:
	    	data.append(np.mean(dff_traces[:, row['start'] + 6 :row['start'] + 9], axis = 1)) #/ np.mean(dff_traces[:, row['start'] : row['start'] - 30], axis = 1) )
	    except:
	    	data.append(np.mean(dff_traces[:, row['start']+ 6:row['end']], axis = 1))
	stim_array = np.array(stim_array)
	#stim_array = stim_array[:, 0:10]

	data = np.array(data)

	return data, stim_array, im_array

def download_data_v2():
    stimulus = [stim_info.LOCALLY_SPARSE_NOISE]
    
    my_ids = (503820068, 501498760, 505801925, 510706209)
    for exp_id in my_ids:
        boc.get_ophys_experiments(exp_id)
#     data_set = {exp_id:boc.get_ophy_experiment_data(experiment_container_ids = exp_id, stimuli = stimulus) for exp_id in my_ids}
    return data_set
    
def download_data(region, cre_line = None, stimulus = None):
	'''
	region = [reg1, reg2, ...]
	cre_line = [line1, line2, ...]
	'''
	boc = BrainObservatoryCache(manifest_file='boc/manifest.json')
	ecs = boc.get_experiment_containers(targeted_structures=region)#, cre_lines=cre_line)

	ec_ids = [ ec['id'] for ec in ecs ]

	exp = boc.get_ophys_experiments(experiment_container_ids=ec_ids)

	if stimulus == None:
		exp = boc.get_ophys_experiments(experiment_container_ids=ec_ids)

	else:
		exp = boc.get_ophys_experiments(experiment_container_ids=ec_ids, stimuli = stimulus)


	exp_id_list = [ec['id'] for ec in exp]

	data_set = {exp_id:boc.get_ophys_experiment_data(exp_id) for exp_id in exp_id_list}

	return data_set 

def get_data(data_set, stimulus):
	'''
	returns dff, images, stim_table
	'''

	time, dff_traces = data_set.get_dff_traces()

	try:
		images = data_set.get_stimulus_template(stimulus)
	except:
		print "No stimulus template..."
		images = None

	stim_table = data_set.get_stimulus_table(stimulus)

	return dff_traces, images, stim_table

In [3]:
boc = BrainObservatoryCache(manifest_file='boc/manifest.json')

regions = boc.get_all_targeted_structures()
# lines =  ['Cux2-CreERT2']#, 'Rbp4-Cre', 'Rorb-IRES2-Cre'] 


#this dataset just has the locally sparse noise data from layer 2/3 of primary visual cortex
data_set = download_data(regions, [], [stim_info.LOCALLY_SPARSE_NOISE])

In [4]:
# experiment container: same target imaging area across specified experiments

#these are the IDs of the different datasets that have locally sparse noise 
#stimulus data, in layer 2/3 of primary visual cortex

dsi =data_set[501839084]
for ikey in data_set.keys():
    dsi = data_set[ikey]
    print(dsi.get_metadata()['targeted_structure'])

VISal
VISl
VISp
VISp
VISp
VISpm
VISp
VISp
VISp
VISp
VISal
VISp
VISp
VISl
VISp
VISp
VISp
VISpm
VISp
VISal
VISp
VISpm
VISp
VISpm
VISal
VISp
VISp
VISpm
VISpm
VISal
VISp
VISpm
VISl
VISl
VISp
VISp
VISl
VISpm
VISal
VISal
VISp
VISp
VISpm
VISpm
VISp
VISal
VISp
VISl
VISal
VISp
VISal
VISp
VISl
VISpm
VISl
VISpm
VISl
VISp
VISp
VISpm
VISp
VISp
VISp
VISp
VISp
VISp
VISpm
VISp
VISp
VISpm
VISp
VISpm
VISl
VISp
VISl
VISp
VISp
VISp
VISp
VISp
VISp
VISal
VISp
VISp
VISl
VISp
VISl
VISp
VISp
VISpm
VISal
VISpm
VISp
VISpm
VISal
VISl
VISal
VISp
VISp
VISp
VISl
VISpm
VISp
VISal
VISal
VISpm
VISl
VISp
VISp
VISp
VISp
VISal


In [19]:
from allensdk.brain_observatory.locally_sparse_noise import LocallySparseNoise


#here we get the RF's that allen computed, the stimulus images, and the dff data. 
#This takes a while to run, which is why I'm only doing it for one of the datasets 
#(which you can roughly think of as being a single mouse)

key = data_set.keys()[0]
print(key)

510524416


In [ ]:
    # compile these two tables
    # LOOK UP TABLES:
        # region2container, table of columns:
            # container_id, mouse_id, regions
        # container2cell, table of columns:
            # container_id, cell_id

In [ ]:
dsi.

In [ ]:
#the line below literally takes forever, only uncomment this if you really care about the lsn stimulus
# lsn  = LocallySparseNoise(data_set[key])
# http://alleninstitute.github.io/AllenSDK/_modules/allensdk/brain_observatory/locally_sparse_noise.html
# """ Perform tuning analysis specific to the locally sparse noise stimulus.

dff, images, stim_table = get_data(data_set[key], stim_info.LOCALLY_SPARSE_NOISE)


In [ ]:
dsi = data_set[500855614] # nwb_data_set class

In [ ]:
dsi.FILE_METADATA_MAPPING

In [ ]:
dsi.get_metadata()

In [ ]:
dsi.list_stimuli()

In [ ]:
print(dsi.STIMULUS_TABLE_TYPES)

In [ ]:
print(dff.shape, images.shape)
# cells, traces?, 

In [ ]:
print(list(stim_table))
print(type(stim_table))

In [ ]:
#here we plot the receptive fields which AIBS computed
# WHAT IS THIS COMPARISON
# receptive_field: space x cell by on/off responsiveness, need to verify this!!

n_cells = lsn.receptive_field.shape[2] # 

fig = plt.figure(figsize = [10, 100] )
i = 1
for n in range(n_cells):
    ax = plt.subplot(np.ceil(n_cells/ 2.) + 1, 2, i)
    
    rf = np.concatenate((lsn.receptive_field[:,:,i -1, 0], np.zeros([lsn.receptive_field.shape[0], 10]), lsn.receptive_field[:,:,i -1, 1]), axis = 1)

    plt.imshow(rf, 'PuRd')
    plt.axis('off')

    i+=1


In [ ]:
#here's where we get the tensors which are useful for fitting models
data, stim_array, _ = arrange_data_glm(dff, images, stim_table)

In [ ]:
#data is n_timepoints by n_neurons. stim_array is n_timepoints by im_y by im_x
# QUESTION: WHAT DOES TIME MEAN IN THIS CASE... WHY ARE THESE VALS SELECTED? what is the time block that we selected? 

print data.shape, stim_array.shape
# guessing that data represents the average(?) df/f over our given time bin

In [ ]:
nplt = 8
for i in range(nplt):
    ax = plt.subplot(np.ceil(nplt) + 1, 2, i+1)
    plt.imshow(stim_array[i], cmap = 'Greys')
    plt.axis('off')

# image presented over a particular time bin

In [ ]:
# dataset definition
    # target area, imaging depth, cre line
    # experimental sessions of region
    # only one session has locally sparse noise (session C)
# desire:
    # PROCESSING PIPELINE:
        # for every cell... associate these variables:
            # Stimuli.xTraining: each image 
            # QUESTION: IS LOCALLY SPARSE NOISE SAME OR DIFFERENT SET OF IMAGES ACROSS EXPERIMENTS? 
            # ytraining: associated calcium rate
                # convert calcium to spiking via pfdd (w/e that pkg is)

# DEFINING AND ORGANIZING CELLS OF INTEREST
    # STATICS
    # lines =  ['Cux2-CreERT2']#, 'Rbp4-Cre', 'Rorb-IRES2-Cre'] # does cell line differ among regions? 
    
    # SWEEPS
    # regions = ['VISp']#, 'VISpm', 'VISl', 'VISal'] # need to figure out which
    
    # only download for locally sparse noise...
    # data_set = download_data(regions, lines, [stim_info.LOCALLY_SPARSE_NOISE])
    
    # compile these two tables
    # LOOK UP TABLES:
        # region2container, table of columns:
            # container_id, mouse_id, regions
        # container2cell, table of columns:
            # container_id, cell_id

# PROCESS PER CELL
# dff2y -conv_dff2y.py-> y -gather_xy.py-> xy -apply_ald.m-> rf

# PYTHON OUTPUTS
# /dff-container_id-cell_id.mat  (x)
    # dff: trace for given cell (get output from dff or data via glm)
    # container_id
    # cell_id
    
# /y-container_id-cell_id.mat
    # y: spike rate per image 
    # container_id
    # cell_id

# /img-container_id.mat
    # x: image per trace
    # container_id
    
# MATLAB LOADING
# xy_loader.m
    # [x y] = xy_loader(container_id, cell_id)
    # Desc: loads the relevant image and spike rate for a given cell
    # x: taken from img-container_id.mat
    # y: taken from y_container_id-cell_id.mat

# MATLAB TO DIR
# /rf-container_id-cell_id.mat
    # img: xy image of derived receptive field
    # container_id
    # cell_id

# GENERAL OBJECTIVE [obsolete]
    # convert df/f to spike rate
    # write out to matlab readable file
    

In [5]:
def extract_from_data_set(data_set, key, extract = False):
    # our data set
    dsi = data_set[key]

    # get the data and stim array
    data = []
    stim_array = []
    if extract:
        dff, images, stim_table = get_data(dsi, stim_info.LOCALLY_SPARSE_NOISE)
        data, stim_array, _ = arrange_data_glm(dff, images, stim_table)
    
    # store metadata
    meta = {
        'experiment_container_id': dsi.get_metadata()['experiment_container_id'],
        'ophys_experiment_id': dsi.get_metadata()['ophys_experiment_id'],
        'session_type': dsi.get_session_type(),
        'cell_specimen_ids': dsi.get_cell_specimen_ids(),
        'cell_specimen_indices': dsi.get_cell_specimen_indices(dsi.get_cell_specimen_ids()),
        'targeted_structure': dsi.get_metadata()['targeted_structure'],
        'num_cells':np.max(dsi.get_cell_specimen_indices(dsi.get_cell_specimen_ids()))+1
    }
    return data, stim_array, meta

In [21]:
data, stim_array, meta = extract_from_data_set(data_set, data_set.keys()[0])

In [6]:
extract_now = True
meta_list = []
for i, ikey in enumerate(data_set.keys()):
    mdff_array, stim_array, meta = extract_from_data_set(data_set, ikey, extract_now)

    for icell in range(meta['num_cells']):
        ifname = "mdff_%s_%s.mat"%(meta['experiment_container_id'],icell)

        dff_dict = {'mdff': mdff_array[:,icell], 
                    'experiment_container_id': meta['experiment_container_id'],
                    'cell_rel_idx': meta['cell_specimen_indices'][icell],
                    'cell_idx': meta['cell_specimen_ids'][icell]}

        # write out cell dff data
        wrider = './data/mdff/%s/'%(meta['experiment_container_id'])
        try:
            os.mkdir(wrider)
        except:
            print('already exists!')
        scipy.io.savemat(wrider+ifname, dff_dict)

    # write out image data
    sfname = 'img_%s.mat'%(meta['experiment_container_id'])
    stim_dict = {'img': stim_array,
                  'experiment_container_id': meta['experiment_container_id']}
    scipy.io.savemat('./data/img/' + sfname, stim_dict)
    
    # gather meta data
    meta.pop('cell_specimen_indices')
    meta.pop('cell_specimen_ids')
    meta_list.append(meta)

# collect metadata into a csv
meta_list = pd.DataFrame.from_dict(meta_list)
meta_list.to_csv('./data/meta.csv')

already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already exists!
already 

In [9]:
import scipy.io

In [13]:
len(data_set.keys())

17